In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns

In [86]:
# Chargement des données
df1 = pd.read_csv('Population_2000_2018.csv')
df2 = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
df3 = pd.read_csv('pib.csv')
df4 = pd.read_csv('completedispo.csv')

In [87]:

# Filtrer les données pour ne garder que l'année 2017
df1 = df1[df1['Année'] == 2017]

# Garder seulement les colonnes souhaitées
df1 = df1[['Zone', 'Élément', 'Unité', 'Valeur']]

# Multiplier la colonne 'Valeur' par 1000 pour obtenir la population totale
df1['Valeur'] = df1['Valeur'] * 1000

# Supprimer la colonne 'Unité'
df1 = df1.drop(columns=['Unité'])

# Supprimer la colonne 'Élément'
df1 = df1.drop(columns=['Élément'])

# Renommer la colonne 'Valeur' en 'Population'
df1.rename(columns={'Valeur': 'Population'}, inplace=True)

In [88]:
df1

,Zone,Population
17,Afghanistan,36296113.0
36,Afrique du Sud,57009756.0
55,Albanie,2884169.0
74,Algérie,41389189.0
93,Allemagne,82658409.0
...,...,...
4333,Venezuela (République bolivarienne du),29402484.0
4352,Viet Nam,94600648.0
4371,Yémen,27834819.0
4390,Zambie,16853599.0


In [89]:
df2.head(5)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [90]:

# Supprimer les colonnes spécifiées
columns_to_drop = ['Code Domaine', 'Domaine', 'Code zone', 'Code Élément', 'Code Produit', 'Code année', 'Année', 'Symbole', 'Description du Symbole']
df2 = df2.drop(columns=columns_to_drop)

# Pivoter les données en utilisant 'Élément' comme colonnes
df2 = df2.pivot_table(index=['Zone', 'Produit'], columns='Élément', values='Valeur', aggfunc='sum').reset_index()

In [91]:
df2

Élément,Zone,Produit,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,Abats Comestible,NaN,NaN,NaN,5.0,1.47,0.19,0.64,53.0,NaN,6.0,53.0,NaN,48.0,0.0,NaN,NaN,NaN
1,Afghanistan,"Agrumes, Autres",NaN,NaN,NaN,1.0,1.32,0.01,0.02,50.0,0.0,33.0,48.0,2.0,17.0,0.0,NaN,NaN,NaN
2,Afghanistan,"Alcool, non Comestible",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0
3,Afghanistan,Aliments pour enfants,NaN,NaN,NaN,1.0,0.10,0.01,0.04,4.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,Ananas et produits,NaN,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16042,Îles Salomon,Viande de Suides,0.0,NaN,NaN,44.0,4.49,4.24,1.38,3.0,NaN,0.0,3.0,NaN,2.0,0.0,NaN,NaN,0.0
16043,Îles Salomon,Viande de Volailles,0.0,NaN,NaN,18.0,4.45,1.31,1.51,3.0,0.0,6.0,3.0,NaN,0.0,0.0,NaN,NaN,3.0
16044,Îles Salomon,"Viande, Autre",0.0,NaN,NaN,0.0,0.01,0.00,0.00,0.0,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
16045,Îles Salomon,Vin,0.0,NaN,NaN,0.0,0.25,0.00,0.00,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0


In [92]:
# Parcourir chaque colonne et remplacer les NaN par 0.0 si la colonne contient uniquement des valeurs numériques
for column in df2.columns:
    if df2[column].dropna().apply(np.isreal).all():
        df2[column].fillna(0.0, inplace=True)


In [93]:
# Grouper par 'Zone' et sommer les valeurs numériques
df2 = df2.groupby('Zone').sum(numeric_only=True).reset_index()


In [94]:
df2

Élément,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,0.0,456.0,94.0,1997.0,357.31,30.63,54.09,15139.0,601.0,4555.0,12968.0,1146.0,11609.0,-111.0,470.0,115.0,426.0
1,Afrique du Sud,0.0,9371.0,1448.0,2987.0,556.42,81.92,83.36,66840.0,10968.0,9020.0,31726.0,3108.0,68455.0,-594.0,212.0,21564.0,-322.0
2,Albanie,0.0,697.0,177.0,3400.0,1260.49,117.57,119.50,4879.0,156.0,884.0,3631.0,321.0,4231.0,-115.0,53.0,113.0,72.0
3,Algérie,0.0,4884.0,2405.0,3345.0,798.71,79.62,92.85,45365.0,743.0,19718.0,33056.0,3683.0,26763.0,-22.0,358.0,993.0,374.0
4,Allemagne,0.0,30904.0,8316.0,3559.0,943.24,147.96,104.07,174960.0,58793.0,66550.0,77988.0,5756.0,167828.0,-831.0,1712.0,51109.0,651.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Émirats arabes unis,0.0,1479.0,2661.0,3370.0,627.67,89.31,107.26,11795.0,4914.0,14986.0,5958.0,525.0,1793.0,94.0,6.0,1068.0,95.0
170,Équateur,0.0,1017.0,2475.0,2585.0,563.62,89.24,65.77,23810.0,8897.0,2134.0,9465.0,860.0,30843.0,-102.0,140.0,9955.0,377.0
171,États-Unis d'Amérique,0.0,151389.0,171964.0,3762.0,990.38,167.00,113.61,849821.0,207278.0,84507.0,322036.0,36551.0,974034.0,2576.0,12380.0,152924.0,1454.0
172,Éthiopie,0.0,4710.0,4126.0,2304.0,361.77,26.80,65.64,55860.0,1259.0,2967.0,38502.0,2915.0,52164.0,-6.0,1038.0,4577.0,-1983.0


In [95]:
# Filtrer les données de df3 pour l'année 2017
df3 = df3[df3['Année'] == 2017]

# Garder seulement les colonnes 'Zone' et 'Valeur' pour df3
df3 = df3[['Zone', 'Valeur']]

# Renommer la colonne 'Valeur' en 'PIB' pour df3
df3.rename(columns={'Valeur': 'PIB'}, inplace=True)

In [96]:
df3

,Zone,PIB
17,Pays-Bas (Royaume des),55088.6
37,Afghanistan,2096.1
59,Afrique du Sud,13950.5
81,Albanie,12771.0
103,Algérie,11809.5
...,...,...
3963,Uruguay,23107.0
3985,Vanuatu,3031.2
4007,Viet Nam,9050.7
4029,Zambie,3395.5


In [97]:
df4.columns

Index(['Code Domaine', 'Domaine', 'Code zone (M49)', 'Zone', 'Code Élément',
       'Élément', 'Code Produit (CPC)', 'Produit', 'Code année', 'Année',
       'Unité', 'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object')

In [98]:

# Supprimer les colonnes spécifiées (ajustées pour df4)
adjusted_columns_to_drop = ['Code Domaine', 'Domaine', 'Code Élément', 'Code année', 'Année', 'Symbole', 'Description du Symbole']
df4 = df4.drop(columns=adjusted_columns_to_drop)

# Pivoter les données en utilisant 'Élément' comme colonnes
df4 = df4.pivot_table(index=['Zone', 'Produit'], columns='Élément', values='Valeur', aggfunc='sum').reset_index()
df4.columns


Index(['Zone', 'Produit', 'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité intérieure', 'Importations - Quantité', 'Nourriture',
       'Production', 'Variation de stock'],
      dtype='object', name='Élément')

In [99]:
# Remplacer les NaN par 0.0 pour les colonnes contenant uniquement des valeurs numériques
for column in df4.columns:
    if df4[column].dropna().apply(np.isreal).all():
        df4[column].fillna(0.0, inplace=True)

# Grouper par 'Zone' et sommer les valeurs numériques
df4 = df4.groupby('Zone').sum(numeric_only=True).reset_index()

In [100]:
df2

Élément,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,0.0,456.0,94.0,1997.0,357.31,30.63,54.09,15139.0,601.0,4555.0,12968.0,1146.0,11609.0,-111.0,470.0,115.0,426.0
1,Afrique du Sud,0.0,9371.0,1448.0,2987.0,556.42,81.92,83.36,66840.0,10968.0,9020.0,31726.0,3108.0,68455.0,-594.0,212.0,21564.0,-322.0
2,Albanie,0.0,697.0,177.0,3400.0,1260.49,117.57,119.50,4879.0,156.0,884.0,3631.0,321.0,4231.0,-115.0,53.0,113.0,72.0
3,Algérie,0.0,4884.0,2405.0,3345.0,798.71,79.62,92.85,45365.0,743.0,19718.0,33056.0,3683.0,26763.0,-22.0,358.0,993.0,374.0
4,Allemagne,0.0,30904.0,8316.0,3559.0,943.24,147.96,104.07,174960.0,58793.0,66550.0,77988.0,5756.0,167828.0,-831.0,1712.0,51109.0,651.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Émirats arabes unis,0.0,1479.0,2661.0,3370.0,627.67,89.31,107.26,11795.0,4914.0,14986.0,5958.0,525.0,1793.0,94.0,6.0,1068.0,95.0
170,Équateur,0.0,1017.0,2475.0,2585.0,563.62,89.24,65.77,23810.0,8897.0,2134.0,9465.0,860.0,30843.0,-102.0,140.0,9955.0,377.0
171,États-Unis d'Amérique,0.0,151389.0,171964.0,3762.0,990.38,167.00,113.61,849821.0,207278.0,84507.0,322036.0,36551.0,974034.0,2576.0,12380.0,152924.0,1454.0
172,Éthiopie,0.0,4710.0,4126.0,2304.0,361.77,26.80,65.64,55860.0,1259.0,2967.0,38502.0,2915.0,52164.0,-6.0,1038.0,4577.0,-1983.0


In [101]:
df4

Élément,Zone,Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Importations - Quantité,Nourriture,Production,Variation de stock
0,Afghanistan,4605.78,393.23,80.06,119.02,17175.00,6112.00,14274.00,11894.00,207.00
1,Afrique du Sud,5804.14,531.35,177.18,159.38,61423.98,8598.84,30290.10,68150.05,4086.01
2,Albanie,6652.45,1168.31,237.66,229.43,4868.07,776.40,3368.90,4235.70,-26.00
3,Algérie,6766.34,747.90,194.25,177.25,43823.71,18102.15,30953.66,27248.83,777.00
4,Allemagne,7083.84,841.88,292.42,208.09,175238.97,64024.21,69590.33,166934.41,3730.01
...,...,...,...,...,...,...,...,...,...,...
175,Émirats arabes unis,6149.52,482.45,221.10,164.69,11692.18,15693.74,4576.90,1995.87,616.00
176,Équateur,5120.47,577.16,194.57,125.58,24431.00,2129.29,9688.08,30957.90,-373.00
177,États-Unis d'Amérique,7556.36,963.44,335.94,226.51,830357.65,82992.78,313199.30,947010.27,3959.00
178,Éthiopie,4969.90,369.87,57.98,147.00,53835.36,3259.12,39351.35,52989.13,1368.00


In [102]:
# Fusionner df1, df3, et df4 en utilisant df4 comme base
merged_df_based_on_df4 = df4.merge(df1, on='Zone', how='right', indicator='Merge_4_1').merge(df3, on='Zone', how='right', indicator='Merge_4_1_3')

# Filtrer pour ne garder que les lignes où Merge_4_1 et Merge_4_1_3 sont "both"
filtered_merged_df = merged_df_based_on_df4[(merged_df_based_on_df4['Merge_4_1'] == 'both') & 
                                            (merged_df_based_on_df4['Merge_4_1_3'] == 'both')]

# Supprimer les colonnes d'indicateur
filtered_merged_df = filtered_merged_df.drop(columns=['Merge_4_1', 'Merge_4_1_3'])


In [104]:
# Exporter le dataframe vers un fichier CSV
filtered_merged_df.to_csv('cleandf.csv', index=False)